In [1]:
from socket import *
from config import *
import json
import numpy as np

In [6]:
class SnakeGame(object):
    def __init__(self, seed_num=0):
        super(SnakeGame, self).__init__()
        self.seed_num = seed_num
        print(f"种子号为 {SEED_LIST[seed_num]}")
        self.conn = socket(AF_INET, SOCK_STREAM)
        self.steps = 0
        self.route = ''
        self.directions = ['w', 'a', 's', 'd']
        self.head_abs = (0, 0)
        self.head_view = []
        
    def __del__(self):
        self.close()
        
    def connect(self, host=HOST, port=PORT):
        try:
            self.conn.connect((host, port))
        except ConnectionRefusedError:
            print("请先允许客户端文件 subg.client")
        else:
            print("已经登入了种子岛")

    def pretty_print(self, view):
        h = len(view)
        w = len(view[0])
        
        for i in range(h):
            line = [emoji_table[n] for n in view[i]]
            line_print = "".join(line)
            print(line_print)
                
    def look_around(self):
        self.conn.send(b'g')
        o = self.conn.recv(4096)
        if not o:
            print("游戏结束")
            self.summary()
            return -1, -1, []
        else:
            x, y, view = json.loads(o.decode('utf-8')).values()
            print(f"当前蛇头在({x}, {y})")
            self.pretty_print(view)
            self.head_abs = (x, y)
            self.head_view = np.array(view)

    def move(self, move_char):
        if move_char in self.directions:
            self.steps += 1
            self.route += move_char
            self.conn.send(bytes(move_char, encoding='utf-8'))
            
        else:
            print(f"Invalid move Command! ({move_char} not in 'wasd')")
        
    
    def wander(self, wander_str, after_see=False):
        # check whether wander_str is valid
        for move_char in wander_str:
            if move_char not in self.directions:
                print(f"Invalid move Command! ({move_char} not in 'wasd')")
                break
        
        for move_char in wander_str:
            self.move(move_char)
        if after_see:
            self.look_around()
            
    def walk_bydir(self, direction='d', distance=1):
        """walk by one direction for 5 distance
        if next step is DEATH, then take the vertical direction
        just use the **first** view
        """    
        head_rel = self.find_head_rel()
        route = ''
            
        print(f"Now take step to Dir {direction} for {distance} distance")
        
        if direction not in self.directions:
            print(f"Invalid direction! ({direction} not in 'wasd')")
        else:
            dir_num = self.directions.index(direction)
            ver_direction = [self.directions[i] for i in range(4) if i != dir_num and i != (dir_num + 2) % 4]
            
            print(ver_direction)
            
            d = ''
            i = 0
            while i < distance:
                if self.head_view[set_add(head_rel, dir_xy[direction])] not in DEATH:
                    d = direction
                    i += 1
                elif self.head_view[set_add(head_rel, dir_xy[ver_direction[0]])] not in DEATH:
                    d = ver_direction[0]
                elif self.head_view[set_add(head_rel, dir_xy[ver_direction[1]])] not in DEATH:
                    d = ver_direction[1]
                else:
                    print("Dead End WARNING!!!")
                    break
                    
#                 print(f"d is {d}")
#                 print(f"Head: {head_rel}  xychange: {dir_xy[d]}")
#                 print(f"Next Head is {set_add(head_rel, dir_xy[d])}")
                    
                head_rel = set_add(head_rel, dir_xy[d])
                route += d
                
        return route
            
    def find_head_rel(self):
        head_x_rel, head_y_rel = np.where(self.head_view == HEAD)
        head_x_rel = head_x_rel[0]
        head_y_rel = head_y_rel[0]
        return (head_x_rel, head_y_rel)
    
    def summary(self):
        print(f'The route is {self.route}')
        print(f"All steps is {self.steps}")
        return self.steps, self.route
        
    def close(self):
        self.conn.close()

In [7]:
try:
    if game:
        game.close()
except NameError:
    pass
game = SnakeGame()
game.connect()

种子号为 2020
已经登入了种子岛


In [8]:
game.look_around();

当前蛇头在(1, 1)
🍄🍄🍄🍄🍄🍄
🍄🐉⬜⬜🍄🍄
🍄⬜⬜⬜⬜⬜
🍄⬜❤️🍄⬜⬜
🍄⬜⬜🍄⬜⬜
🍄💞⬜⬜⬜🍄


In [9]:
game.walk_bydir('d', distance=4)

Now take step to Dir d for 4 distance
['w', 's']
0
d is d
Head: (1, 1)  xychange: (0, 1)
Next Head is (1, 2)
0
d is d
Head: (1, 2)  xychange: (0, 1)
Next Head is (1, 3)
d is s
Head: (1, 3)  xychange: (1, 0)
Next Head is (2, 3)
0
d is d
Head: (2, 3)  xychange: (0, 1)
Next Head is (2, 4)
0
d is d
Head: (2, 4)  xychange: (0, 1)
Next Head is (2, 5)


'ddsdd'

In [32]:
game.head_view[(1, 4)]

1

In [30]:
game.head_view

array([[ 1,  1,  1,  1,  1,  1],
       [ 1, 12,  0,  0,  1,  1],
       [ 1,  0,  0,  0,  0,  0],
       [ 1,  0,  5,  1,  0,  0],
       [ 1,  0,  0,  1,  0,  0],
       [ 1,  6,  0,  0,  0,  1]])

In [19]:
game.find_head_rel()

(1, 1)

In [43]:
x, y, view = game.look_around()

当前蛇头在(6, 6)
⬜⬜⬜⬜⬜❤️⬜⬜⬜
⬜🍄⬜⬜⬜⬜❤️⬜⬜
⬜🍄⬜⬜⬜🍄⬜⬜⬜
⬜⬜⬜🍄⬜⬜⬜🍄⬜
🍄🍄🐍🐍🐉🍄⬜⬜⬜
❤️🍄🍄⬜🍄⬜💞⬜❤️
⬜⬜⬜⬜🍄🍄⬜⬜⬜
🍄⬜🍄⬜⬜⬜⬜⬜⬜
❤️🍄⬜⬜⬜⬜⬜⬜⬜


In [48]:
game.wander('s', after_see=True)

当前蛇头在(5, 4)
🍄⬜⬜⬜🍄🍄🍄⬜⬜
🍄⬜⬜⬜⬜⬜⬜❤️⬜
🍄⬜⬜🍄⬜⬜⬜⬜❤️
🍄⬜⬜🍄🐍🐍⬜🍄⬜
🍄💞⬜⬜🐉🍄⬜⬜⬜
🍄⬜🍄🍄⬜⬜⬜🍄⬜
🍄⬜❤️🍄🍄⬜🍄⬜💞
🍄🍄⬜⬜⬜⬜🍄🍄⬜
🍄⬜🍄⬜🍄⬜⬜⬜⬜


In [14]:
import numpy as np

In [33]:
view_array = np.array(view)
head_x_rel, head_y_rel = np.where(view_array == 12)
head_x_rel = head_x_rel[0]
head_y_rel = head_y_rel[0]



In [31]:
x_rel

array([4], dtype=int64)